In [1]:
from llama_index.llms.ollama import Ollama
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.embeddings import resolve_embed_model
from langchain_community.document_loaders import PyPDFLoader
import json

In [2]:
PDF_DATA_PATH = "./pdf_reports/"

## Initialize the Ollama model

In [3]:
llm = Ollama(model="llama3:8b", request_timeout=1200.0)

## Ask questions to base model

In [4]:
result = llm.complete("In the first quarter of 2023, how much did the consolidated gross revenue increased of DigiPlus Interactive Corp?")
result.text

'I\'m just an AI, I don\'t have access to real-time or historical financial data. Additionally, I couldn\'t find any information on a company called "DigiPlus Interactive Corp". It\'s possible that it\'s a private company or not publicly traded, which would explain the lack of publicly available financial information.\n\nIf you\'re looking for financial information on a specific company, I recommend checking reputable sources such as the company\'s website, annual reports, or financial databases like EDGAR (for US-listed companies) or SEDAR (for Canadian-listed companies). You can also try contacting the company directly to request the information.'

In [5]:
result = llm.complete("What is the cash balance as of March 31, 2024 of DigiPlus Interactive Corp?")
result.text

"I'm just an AI, I don't have access to real-time financial information or specific company data. However, I can suggest some ways for you to find the answer:\n\n1. Check the company's website: You can visit DigiPlus Interactive Corp's official website and look for their investor relations or financial reports section. They might have posted their latest quarterly or annual report, which would include the cash balance as of March 31, 2024.\n2. Search online databases: You can try searching online databases such as EDGAR (Electronic Data Gathering, Analysis, and Retrieval) or SEDAR (System for Electronic Document Analysis and Retrieval), which provide access to publicly filed documents by companies listed on US or Canadian stock exchanges.\n3. Contact the company directly: You can reach out to DigiPlus Interactive Corp's investor relations department or financial team and ask them about their cash balance as of March 31, 2024. Keep in mind that they might not be able to provide you with

In [6]:
result = llm.complete("How much is the approved cash dividends to all stockholders on March 19, 2024 of DigiPlus Interactive Corp?")
result.text

"I'm not aware of any information about DigiPlus Interactive Corp or its dividend payments. It's possible that you may be thinking of a different company or that this information hasn't been publicly disclosed yet.\n\nAs a reliable and accurate assistant, I can only provide information that is publicly available and up-to-date. If you're looking for information on a specific company's dividend payments, I recommend checking the company's website, contacting their investor relations department, or reviewing financial news sources such as Bloomberg or Reuters."

## Implement Retrival-Augmented Generation (RAG)

In [7]:
documents = SimpleDirectoryReader(PDF_DATA_PATH).load_data()

In [8]:
embed_model = resolve_embed_model("local:BAAI/bge-m3")

C:\Users\Tools\Desktop\analyze_financial_reports\analyze_reports_venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
vector_index = VectorStoreIndex.from_documents(documents, embed_model=embed_model)

In [10]:
query_engine = vector_index.as_query_engine(llm=llm)

In [11]:
def ask_llm(question):
    result = query_engine.query(question)
    return result.response

## Ask questions to RAG model

In [12]:
ask_llm("In the first quarter of 2023, how much did the consolidated gross revenue increased of DigiPlus Interactive Corp?")

'₱3,248.7 million.'

In [13]:
ask_llm("What is the cash balance as of March 31, 2024 of DigiPlus Interactive Corp?")

'According to the unaudited interim condensed consolidated statement of cash flows for the six months ended June 30, 2023, the cash and cash equivalents at end of period was 3,446,995. \n\nSince we are only given information up to June 30, 2023, there is no way to determine the cash balance as of March 31, 2024.'

In [14]:
ask_llm("How much is the approved cash dividends to all stockholders on March 19, 2024 of DigiPlus Interactive Corp?")

'The approved cash dividend to all stockholders of DigiPlus Interactive Corp. on March 19, 2024 is P0.18 per outstanding common share.'